In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

%tensorflow_version 2.x
print(tf.__version__)

Found GPU at: /device:GPU:0
TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.0


In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.3 GB  | Proc size: 455.1 MB
GPU RAM Free: 11372MB | Used: 69MB | Util   1% | Total 11441MB


In [0]:
import numpy as np
import argparse
import time
import cv2
import os
from glob import glob
import json

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/My\ Drive/Colab\ Notebooks/HW_3/darknet/data

/content/drive/My Drive/Colab Notebooks/HW_3/darknet/data


In [0]:
labelsPath = os.path.sep.join(['yolo-test-2', "obj.names"])
LABELS = open(labelsPath).read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

weightsPath = os.path.sep.join(['yolo-test-2', "yolo-obj_best.weights"])
configPath = os.path.sep.join(['yolo-test-2', "yolo-obj.cfg"])

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [7]:
%%timeit
with tf.device('/gpu:0'):
  image = cv2.imread('valid/1.png')

  (H, W) = image.shape[:2]
  ln = net.getLayerNames()
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (512, 256),
          swapRB=True, crop=False)

  net.setInput(blob)
  layerOutputs = net.forward(ln)

  boxes = []
  confidences = []
  classIDs = []

  for output in layerOutputs:
    for detection in output:
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      if confidence > 0.1:
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")

        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))

        boxes.append([x, y, int(width), int(height)])
        confidences.append(float(confidence))
        classIDs.append(classID)
                                  
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)

    if len(idxs) > 0:
      for i in idxs.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        color = [int(c) for c in COLORS[classIDs[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, color, 2)

1 loop, best of 3: 1.07 s per loop
